In [7]:
import pickle
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

In [8]:
from pathlib import Path

# Load your filenames
filenames = pickle.load(open('filenames.pkl','rb'))

# Convert Windows-style paths to Linux-compatible paths
fixed_filenames = []
for f in filenames:
    f = f.replace("\\", "/")  # fix backslashes
    if f.lower().startswith("d:/"):  # remove Windows drive prefix
        f = f[3:]
    fixed_path = Path("/home/anas/BollywoodCeleb_Classifier/data") / "/".join(f.split("/")[-2:])
    fixed_filenames.append(fixed_path)

# Replace the old filenames with the fixed ones
filenames = fixed_filenames

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [9]:
def feature_extractor(img_path, model):
    # Convert to Path object for cross-platform compatibility
    img_path = Path(img_path)

    # Check if the file exists before loading
    if not img_path.exists():
        raise FileNotFoundError(f"Image not found at: {img_path.resolve()}")

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img)

    # Extract features
    result = model.predict(preprocessed_img, verbose=0).flatten()

    return result

In [ ]:
features = []
for file in tqdm(filenames):
    features.append(feature_extractor(file, model))


In [ ]:
pickle.dump(features,open('feature_embedding.pkl','wb'))